In [6]:
from openai import AsyncOpenAI

client = AsyncOpenAI(api_key="1989", base_url="http://localhost:8000/v1")

completion = await client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    messages=[{"role": "user", "content": "What is the capital of France?"}],
)

completion.choices[0].message.content

'<think>\nOkay, so I need to figure out the capital of France. I remember the French alphabet and some of my geography shots from school, but I\'m not entirely sure about the specific capitals. Let me think about the places I know.\n\nI know that France is a big island in the Atlantic Ocean, so the capital should be a major city in that region. I think Toulouse is a town, but I\'m not certain about the spelling. Maybe it\'s "Tozzy"? I recall seeing "Tozzy" in a TV show or movie once, so perhaps it\'s spelled with an \'e\'. So the capital might be called Tozzy.\n\nThen there\'s Paris, which I\'ve heard of too. Paris is the capital of France, but it\'s the capital of several countries too, like Spain and Italy. So Paris is definitely one, but the main one is Toulouse. \n\nI think I heard something about "El B Toll" in TV shows, also known as "Elton John," so I\'m pretty sure the capital is Toulouse. That name kinda reminds me of "Elton John" because it has a similar "ol" sound. \n\nLet m

In [ ]:
# ssh -L 8000:localhost:8000 atreides

In [7]:
import json

puzzles = json.load(open("./data/temporal-clue-puzzles.json"))

In [9]:
chat_completion = await client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?",
        }
    ],
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    logprobs=True
)

chat_completion.choices[0].logprobs

APIConnectionError: Connection error.

In [24]:
[
    token_logprob
    for token_logprob in chat_completion.choices[0].logprobs.content
    if token_logprob.bytes == [60, 47, 116, 104, 105, 110, 107, 62]
]

[ChatCompletionTokenLogprob(token='token_id:151649', bytes=[60, 47, 116, 104, 105, 110, 107, 62], logprob=-7.986990567587782e-06, top_logprobs=[])]

In [19]:
from openai.types.chat.chat_completion import ChatCompletion, Choice, ChoiceLogprobs
from openai.types.chat.chat_completion_message import (
    ChatCompletionMessage,
    FunctionCall,
)
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
    Function,
)


async def stream_completion(prompt: str, n: int = 1) -> ChatCompletion:
    chat_completion: ChatCompletion | None = None
    async for chunk in await client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        stream=True,
        logprobs=True,
        n=n,
    ):
        print(chunk.choices[0].delta.content, end="", flush=True)
        if chat_completion is None:
            chat_completion = ChatCompletion(
                id=chunk.id,
                choices=[
                    Choice(
                        finish_reason="stop",
                        index=choice.index,
                        logprobs=(ChoiceLogprobs() if choice.logprobs else None),
                        message=ChatCompletionMessage(role="assistant"),
                    )
                    for choice in chunk.choices
                ],
                created=chunk.created,
                model=chunk.model,
                object="chat.completion",
            )
        for choice, chunk_choice in zip(chat_completion.choices, chunk.choices):
            choice.finish_reason = chunk_choice.finish_reason or "stop"
            if chunk_choice.logprobs:
                if choice.logprobs is None:
                    choice.logprobs = ChoiceLogprobs()
                if chunk_choice.logprobs.content:
                    if choice.logprobs.content is None:
                        choice.logprobs.content = []
                    choice.logprobs.content.extend(chunk_choice.logprobs.content)
                if chunk_choice.logprobs.refusal:
                    if choice.logprobs.refusal is None:
                        choice.logprobs.refusal = []
                    choice.logprobs.refusal.extend(chunk_choice.logprobs.refusal)
            if chunk_choice.delta.content:
                if choice.message.content is None:
                    choice.message.content = ""
                choice.message.content += chunk_choice.delta.content
            if chunk_choice.delta.refusal:
                if choice.message.refusal is None:
                    choice.message.refusal = ""
                choice.message.refusal += chunk_choice.delta.refusal
            if chunk_choice.delta.function_call:
                if choice.message.function_call is None:
                    choice.message.function_call = FunctionCall(arguments="", name="")
                choice.message.function_call.name += (
                    chunk_choice.delta.function_call.name or ""
                )
                choice.message.function_call.arguments += (
                    chunk_choice.delta.function_call.arguments or ""
                )
            if chunk_choice.delta.tool_calls:
                if choice.message.tool_calls is None:
                    choice.message.tool_calls = []
                for tool_call in chunk_choice.delta.tool_calls:
                    if not tool_call.index in range(len(choice.message.tool_calls)):
                        choice.message.tool_calls.append(
                            ChatCompletionMessageToolCall(
                                id="",
                                function=Function(arguments="", name=""),
                                type="function",
                            )
                        )
                    choice.message.tool_calls[tool_call.index].id += tool_call.id or ""
                    choice.message.tool_calls[tool_call.index].function.name += (
                        tool_call.function.name or "" if tool_call.function else ""
                    )
                    choice.message.tool_calls[tool_call.index].function.arguments += (
                        tool_call.function.arguments or "" if tool_call.function else ""
                    )
        chat_completion.service_tier = chunk.service_tier
        chat_completion.system_fingerprint = chunk.system_fingerprint
        chat_completion.usage = chunk.usage
    assert chat_completion is not None
    return chat_completion


await stream_completion(sorted(puzzles, key=lambda x: x["num_clues"])[0]["prompt"])

APIConnectionError: Connection error.

In [ ]:
import atreides

model = atreides.train_model(
    base_model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    train_dataset="capitals/train",
    val_dataset="capitals/val",
)

In [ ]:

from prisma import Prisma

client = AsyncOpenAI()
prisma = Prisma()

await prisma.connect()

datasets = await prisma.dataset.find_many()

print(datasets)

[]


In [4]:
import json

dataset = await prisma.dataset.create(
    {
        "name": "capitals/train",
        "prompts": {
            "create": [
                {
                    "messages": json.dumps(
                        [{"role": "user", "content": "What is the capital of France?"}]
                    ),
                    "metadata": json.dumps({})
                }
            ]
        },
    }
)
dataset

Dataset(id=1, name='capitals/train', prompts=None)

In [8]:
completion = await prisma.completion.create(
    {
        "prompt_id": 1,
        "model": "gpt-4o",
        "results": json.dumps([{"role": "assistant", "content": "Paris"}]),
    }
)
completion

Completion(id=1, prompt_id=1, prompt=None, model='gpt-4o', results='[{"role": "assistant", "content": "Paris"}]', rewards=[Reward(id=1, completion_id=1, completion=None, title='Accuracy', description='1.0 if the capital is correct, 0.0 otherwise', value=1.0, weight=1.0)])

In [7]:
dataset = await prisma.dataset.find_first(include={"prompts": True})
dataset.prompts

[Prompt(id=1, messages='[{"role": "user", "content": "What is the capital of France?"}]', metadata='{}', datasets=None, completions=None)]

In [3]:
import atreides

try:
    train_dataset = await atreides.get_dataset("capitals/train")
except atreides.NotFoundError:
    train_dataset = await atreides.create_dataset(
        name="capitals/train",
        tasks=[
            {
                "messages": [
                    {"role": "user", "content": "What is the capital of France?"}
                ],
                "pattern_rewards": [
                    {
                        "regex_pattern": r"The capital of France is (\w+)",
                        "expected_capture_groups": ["Paris"],
                    },
                ],
            },
        ],
    )

try:
    val_dataset = await atreides.get_dataset("capitals/val")
except atreides.NotFoundError:
    val_dataset = await atreides.create_dataset(
        name="capitals/val",
        tasks=[
            {
                "messages": [
                    {"role": "user", "content": "What is the capital of England?"}
                ],
                "pattern_rewards": [
                    {
                        "regex_pattern": r"The capital of England is (\w+)",
                        "expected_capture_groups": ["London"],
                    },
                ],
            },
        ],
    )